<a href="https://colab.research.google.com/github/simon-m-mudd/smm_teaching_notebooks/blob/master/transience_from_topography/Common_analyses_to_check_for_transience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Common analyses that you can use to check for landscape transience

*This lesson made by Simon M Mudd and last updated 02/10/2025*

## READ THIS FIRST

**To generate the plots in this notebook there are only a few things you should change!**

**Search for the text "Edit code block below for your site" to find where you need to change the code. Do not alter any of the other code blocks**

* You can change your location by adjusting `lower_left` and `upper_right` corner of your site.
* You can update your basin locations.
* You can plot different basin by changing the basin number.
* You can change the end points of your swath.

So by changing a tiny handful of things you should get a fairly extensive set of figures (saved as png, with file names that tell you what they are) for your selected landscape.

## Stuff we need to do if you are in colab (*not required in notable or in the lsdtopotools apptainer/docker containers*)

The assumption for these notebooks is that you are working in a notable container.

So these steps are turned into comments using a `#` symbol before each line.

If you want to work on google colab you will need to delete the `#` symbols

In [ ]:
#!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
#!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

In [ ]:
#!pip install lsdviztools &> /dev/null

## Part 1: Get data

**First step, copy your Opentopography API key into a text file called "my_OT_api_key.txt" and make sure it is in the same file system as this notebook.**

If you don't know how to do that got to the `basic_topography` section and read lesson 1.

Make sure we are using the latest version of `lsdviztools`

In [ ]:
!pip install lsdviztools --upgrade

Double check that the version is correct. For this lesson we need version 0.4.11 or above.

In [ ]:
import lsdviztools
lsdviztools.__version__

We also need to import some stuff

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio


**Next step: make sure the filename of your API key matches the one below.** Filenames are case sensitive. So if your file is "my_OT_API_key.txt" this will not work in the code block below (because api needs to be lower case).

**Then update the latitude and longitude of the lower left and upper right corner of your study area. You can get a latitude and longitude by right clicking on google maps.**

### Edit code block below for your site!

* change latitude and longitude of your `lower_left` and `upper_right` corners.
* Make sure your api filename is correct.



In [ ]:
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdbasemaptools as bmt

lower_left = [37.87521641344407, 15.628317625157045]
upper_right = [38.30554438686832, 16.210748067627133]

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "DEM"
source_name = "COP30"


My_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = lower_left,
                        upper_right_coordinates = upper_right,
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
My_DEM.print_parameters()
My_DEM.download_pythonic()

DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

**Important**: you should always cite your data source. For this data, you should cite:

*European Space Agency, Sinergise (2021). Copernicus Global Digital Elevation Model. Distributed by OpenTopography. https://doi.org/10.5069/G9028PQB. Accessed: 2023-09-26*

## Part 2: Get a hillshade: **you have to do this for the rest of the plotting to work**

We are going to get a hillshade since this is used in all of the plotting routines in `lsdviztools`.

*You only need to execute this code, you don't need to change anything!*

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This prints the image. It will also save a png file.

In [ ]:
DataDirectory = "./"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=True,
                                 size_format="geomorphology",dpi=400,
                                 out_fname_prefix = "hillshade_plot")

print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

## Part 3 Make slope and curvature maps

Topographic gradient and hillslope curvature give some indications of how quickly landscapes are eroding.

The analyses you use below are calculated by fitting a polynomial surface to topography and calculating the derivatives of this surface. The method follows Hurst et al (2012):

*Hurst, M.D., Mudd, S.M., Walcott, R., Attal, M., Yoo, K., 2012. Using hilltop curvature to derive the spatial distribution of erosion rates. J. Geophys. Res. 117, 2011JF002057. https://doi.org/10.1029/2011JF002057*



You can say that you implemented this algorithm using lsdtopotools version 0.9. The citation is:

*Mudd, S.M., Clubb, F.J., Grieve, S.W.D., Milodowski, D.T., Gailleton, B., Hurst, M.D., Valters, D.V., Wickert, A.D., & Hutton, E.W.H. (2023). LSDtopotools/LSDTopoTools2: LSDTopoTools2 v0.9 (v0.9). Zenodo. https://doi.org/10.5281/zenodo.8076231*

This part runs the analysis:

In [ ]:
# IMPORTANT all the parameter values must be passed as strings.
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"surface_fitting_radius" : "60",
                    "print_slope" : "true",
                    "print_curvature" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The next two code blocks print the gradient:

In [ ]:
%%capture
Base_file = r_prefix

### Plot the topographic gradient
Drape_prefix = r_prefix+"_SLOPE"
this_slope_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="inferno",
                                       cbar_label = "Gradient (m/m)",
                                       save_fig=True, size_format="geomorphology",dpi=400,
                                       out_fname_prefix = "gradient_plot")

In [ ]:
print(this_slope_img)
from IPython.display import display, Image
display(Image(filename=this_slope_img, width=800))

And the next two print the curvature.

In [ ]:
%%capture
Base_file = r_prefix

### Plot the curvature
Drape_prefix = r_prefix+"_CURV"
this_curv_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="RdBu",
                                  cbar_label = "Curvature (1/m)",
                                  cbar_loc = "top",
                                  save_fig=True, size_format="ESURF",dpi=400,
                                  out_fname_prefix = "curvature_plot")

In [ ]:
print(this_curv_img)
from IPython.display import display, Image
display(Image(filename=this_curv_img, width=800))

### We might also want a histogram of topographic gradients

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio

tif_path = r_prefix+"_SLOPE.bil"
with rasterio.open(tif_path) as src:
    data = src.read(1, masked=True)
vals = data.compressed()
vals = vals[np.isfinite(vals)]

# --- (Optional) Convert units for readability ---
# vals = np.degrees(vals)       # if your data are in radians → degrees
# vals = vals * 100             # if your data are m/m → %

# --- Robust clipping to ignore extreme tails (e.g., spikes/noise) ---
lo, hi = np.percentile(vals, [0.5, 99.5])  # tweak as needed
vals_clipped = vals[(vals >= lo) & (vals <= hi)]

# --- Freedman–Diaconis rule for bins (robust width) ---
iqr = np.subtract(*np.percentile(vals_clipped, [95, 5]))
bin_width = 2 * iqr * (len(vals_clipped) ** (-1/3))
if bin_width <= 0:
    bins = 50
else:
    bins = max(10, int(np.ceil((vals_clipped.max() - vals_clipped.min()) / bin_width)))

plt.figure(figsize=(7.5, 4.5))
plt.hist(vals_clipped, bins=bins, color="#2a6fbb", edgecolor="white")
plt.xlabel("Topographic gradient (m/m)")   # set your units
plt.ylabel("Pixel count")
plt.title("Histogram of topographic gradients (0.5–99.5th pct clipped)")
plt.tight_layout()
plt.savefig("slope_histogram.png", dpi=400)

In [ ]:
from IPython.display import display, Image
display(Image(filename="slope_histogram.png", width=800))

## Part 4: Select basins

We can select basins by giving `lsdtopotools` basin outlets.

The below code block allows you to paste in some basin outlets.

So you can change some of the outlets, make sure each outlet has two numbers, separated by a comma, in square brackets.
There needs to be a comma after each of the outlet locations, apart from the last one.
All of these get enclosed in an outer pair of square brackets.

Like this:
```
[ [a,b],
  [c,d],
  [e,f]
  ]
```

**Important**: if your DEM does not contain the entire drainage basin, the basin will not be extracted. That is, you can't get a basin that is beheaded by the edge of your map. Also you should pick an outlet slightly upstream of where you want the basin, for reasons that are too tedious to explain here.

### Edit code block below for your site!

* change the latitude and longitude of your basin outlets using the format described above. The ways you can mess this up are by not having the correct placing of square brackets and having commas missing, or in the wrong place. Follow the format above!

In [ ]:
# Import pandas library
import pandas as pd

data = [ [37.94234239772101, 15.790076672680563],
         [37.93291942111813, 15.792125654538383],
         [38.06143176957989, 16.060604039819015],
         [38.094672987877786, 15.677112531901733],
         [38.20455853467961, 15.67642588617276],
         [38.12546843046537, 16.1230887795859]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("basin_outlets.csv",index=False)
df.head()

This runs the analysis.

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"search_radius_nodes" : "3",
                    "threshold_contributing_pixels" : "500",
                    "print_basin_raster" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "print_chi_data_maps" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This makes the plots:

In [ ]:
#import lsdviztools.lsdmapwrappers as lsdmw
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

#%%capture
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=400, save_fig = True,
                             out_fname_prefix = "basins_plot")

print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

### **The art of basin selection!**

Getting good basins takes a little bit of work. When you put in a basin cooridnate the program
1. Finds a nearby channel (so you need to be close to a channel!).
2. Then finds the nearest upstream *junction* (where two channels come together).
3. Then the default is for the program to follow this downstream to the pixel just above the next downstream junction.
If this all sounds a bit weird it is because the analysis was built to work through junctions of a certain "stream order": you can look that up on the internet if you are interested.

The upshot of all this is that your basin will not be exactly where you placed your point.

If you want to get it close to your point, you need to

1. Control how many channels you have (and therefore how many junctions you have). Do this by changing `threshold_contributing_pixels`.
2. Control how close the program searches for a channel. Do this by changing `search_radius_nodes`


So if your basins are in funny places try playing with those two parameters.

## Part 5: Plot channel profiles

We can look at the channel profiles in your basins.

A typical channel profile will be a plot of elevation against flow distance. But this makes it a little difficult to see changes in channel steepness. Why? Because headwater channels tend to be steeper than downstream channels. In fact, the gradient of channels is thought to scale with drainage area, so to compare relative steepness, you need to control for drainage area.

A clever way to do this is to integrate drainge area as you move upstream. I won't explain why this works by if you want to know about it please read this paper:

*Perron, J.T, and Royden, L. 2013. An Integral Approach to Bedrock River Profile Analysis. Earth Surface Processes and Landforms 38 (6): 570–76. https://doi.org/10.1002/esp.3302.*

This results in a transformed channel coordinate, which we call "chi" (it is this greek letter: $\chi$).

If you plot elevation against $\chi$, you get something that looks like a channel profile. But this profile has the very convinient feature that the gradient of the profile should be proportional to something called the channel steepness index, which in many landscapes is closely correlated with the erosion rate. See:

*Kirby, E., Whipple, K., 2001. Quantifying differential rock-uplift rates via stream profile analysis. Geology 29, 415–418. https://doi.org/10.1130/0091-7613(2001)029<0415:QDRURV>2.0.CO;2*



This first step loads the channel profile data. This came out of the little bit of code you used to get the basins.

In [ ]:
import pandas as pd
channel_file_name = r_prefix+"_chi_data_map.csv"
df = pd.read_csv(channel_file_name)
df.head()

### Edit code block below for your site!

* change the basin number to look at different basins!

In [ ]:
basin = 2

Now we can plot the $\chi$-elevation plot:

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.chi,df_single_basin.elevation,c=df_single_basin.flow_distance)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

chi_profile_name = "chi_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(chi_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

You can also plot the traditional channel profile.

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.flow_distance,df_single_basin.elevation,c=df_single_basin.chi)
plt.xlabel("flow distance$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

channel_profile_name = "channel_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(channel_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

## A swath example

You could collect elevation data along a transect to see trends in elevation. For example, you could look at a transect perpendicular to a river to see what the hillslopes are doing.

However, topographic data is noisy, there are ridges, etc. So an alternative is to get some average representation of the topography along a line.

A *swath profile* takes some baseline (think of this as the middle of your transect) and then looks at elevations around it and calculates the statistic of elevation.

The median values along this *swath* look very much like a single transect, but you can also see how much the elevations vary.

In `lsdtopotools` you can make a swath profile by just telling it the two ends of your transect.

The method is from:

*Hergarten, S., Robl, J., Stüwe, K., 2014. Extracting topographic swath profiles across curved geomorphic features. Earth Surface Dynamics 2, 97–104. https://doi.org/10.5194/esurf-2-97-2014*


### Edit code block below for your site!

* Update the two lat-long pairs below to get the ends of your swath profile

In [ ]:
# Import pandas library
import pandas as pd

data = [ [38.071180814769235, 15.821177352750635],
         [38.10306680861067, 15.796114793399402]
       ]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("swath1.csv",index=False)
df.head()

This runs the anaylsis.

In [ ]:
## Get the basins and the channel profile
import lsdviztools.lsdmapwrappers as lsdmw
lsdtt_parameters = {"calculate_swath_profile" : "true",
                    "calculate_swath_along_line" : "true",
                    "swath_point_spacing" : "45",
                    "swath_bin_spacing" : "90",
                    "swath_points_csv" : "swath1.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This prints the swath footprint (that is, where the data comes from on the map).

In [ ]:
### Plot the swath footprint
Base_file = r_prefix
Drape_prefix = Base_file+"_swathval"
DataDirectory = "./"
this_area_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="jet",
                                       cbar_label = "Swath distance (m)",
                                       cbar_loc = "bottom",
                                       save_fig=True, size_format="ESURF",dpi=600,
                                       out_fname_prefix = "swath_footprint_plot")

In [ ]:
print(this_area_img)
from IPython.display import display, Image
display(Image(filename=this_area_img, width=800))

This prints the actual swath profile.

In [ ]:
import lsdviztools.lsdplottingtools.lsdmap_swathplotting as lsdsp
import lsdviztools.lsdmapwrappers as lsdmw
DataDirectory="./"
swathname = r_prefix+"_swath.csv"
imname = lsdmw.SimpleSwath(DataDirectory,swathname, fig_format = "png", dpi = 500, aspect_ratio = 2)

In [ ]:
from IPython.display import display, Image
display(Image(filename=imname, width=800))

## Get the channel steepnesses and plot

In [ ]:
lsdtt_parameters = {"m_over_n" : "0.45",
                    "print_segmented_M_chi_map_to_csv" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!lsdtt_plotchianalysis -dir . -fname DEM_COP30_UTM -all_stacks true -basin_lists "0,1,2,3,4"

In [ ]:
from IPython.display import display, Image
display(Image(filename="./chi_profile_plots/Stacked_1_chi_stacked_chi.png", width=800))


Or we could have a fancier plot

In [ ]:
ksn_img = lsdmw.PrintChiChannelsAndBasins(DataDirectory,Base_file,
                                          ChannelFileName = "DEM_COP30_UTM_MChiSegmented.csv",
                                          add_basin_labels = False, cmap = "viridis",
                                          cbar_loc = "right",
                                          size_format="geomorphology",dpi=600,
                                          fig_format = "png",
                                          plotting_column="m_chi",
                                          colorbarlabel = "$\mathrm{log}_{10} \; \mathrm{of} \; k_{sn}$",
                                          out_fname_prefix = "ksn_plot")

In [ ]:
from IPython.display import display, Image
display(Image(filename="./ksn_plot_chi_channels_and_basins.png", width=800))